In [1]:
import pandas as pd
import numpy as np

In [2]:
X=pd.read_csv('bankniftyoneminutefinal.csv')

In [3]:
X=X.drop('Unnamed: 6', axis=1)

In [5]:
X['date_time']=pd.to_datetime(X['date'] + ' ' + X['time'])

In [4]:
def resample_stock_data(df, timedelta):
    # make a copy
    df = df.copy()

    # convert index to datetime
    df.index = pd.to_datetime(df.index)

    # sort the index (evidently required by resample())
    df = df.sort_index()

    aggregation_dict = {
         'open': 'first', 
         'high': 'max',
         'low': 'min',
         'close': 'last',
         
    }

    rename_dict = {
        'open': 'open',
        'high': 'high',
        'low': 'low',
        'close': 'close',
        
    }


    return (df
      .resample(timedelta)
      .agg(aggregation_dict)
      .rename(columns=rename_dict)
    )

In [7]:
y=X.set_index('date_time')

In [8]:
X_15=resample_stock_data(y,'15min')
X_15=X_15.dropna()

In [10]:
X_15.head()

,open,high,low,close
date_time,,,,
2015-01-09 09:15:00,18845.90,18864.0,18790.20,18815.15
2015-01-09 09:30:00,18815.15,18821.4,18758.50,18771.60
2015-01-09 09:45:00,18774.15,18808.3,18761.10,18802.90
2015-01-09 10:00:00,18802.90,18813.6,18758.50,18781.15
2015-01-09 10:15:00,18781.15,18807.1,18769.45,18803.00


In [9]:
X_d=resample_stock_data(y,'1d')
X_d=X_d.dropna()

In [11]:
X_d.head()

,open,high,low,close
date_time,,,,
2015-01-09,18845.90,18864.00,18479.65,18598.05
2015-01-12,18632.00,18822.05,18530.10,18808.75
2015-01-13,18889.00,18920.65,18642.80,18724.75
2015-01-14,18670.40,18769.65,18517.90,18636.85
2015-01-15,19340.45,19410.40,19061.35,19134.65


In [12]:
X_15['close_prev_day']=np.NaN

In [13]:
ix=X_d.index

In [14]:
my_df  = pd.DataFrame() 
for i in range(len(X_d)):
    x=X_15.loc[str(ix[i].date())].fillna(X_d["close"][i])
    my_df=pd.concat([my_df,x])

In [15]:
my_df.head()

,open,high,low,close,close_prev_day
date_time,,,,,
2015-01-09 09:15:00,18845.90,18864.0,18790.20,18815.15,18598.05
2015-01-09 09:30:00,18815.15,18821.4,18758.50,18771.60,18598.05
2015-01-09 09:45:00,18774.15,18808.3,18761.10,18802.90,18598.05
2015-01-09 10:00:00,18802.90,18813.6,18758.50,18781.15,18598.05
2015-01-09 10:15:00,18781.15,18807.1,18769.45,18803.00,18598.05


In [33]:
my_df['fif_min_open']=np.NaN
my_df['fif_min_high']=np.NaN
my_df['fif_min_low']=np.NaN
my_df['fif_min_close']=np.NaN
data=pd.DataFrame() 
for i in range(len(X_d)):
    dics={'fif_min_high':X_15.loc[str(ix[i].date())]['high'].iloc[0],
          'fif_min_open':X_15.loc[str(ix[i].date())]['open'].iloc[0],
           'fif_min_close':X_15.loc[str(ix[i].date())]['close'].iloc[0],
          'fif_min_low':X_15.loc[str(ix[i].date())]['low'].iloc[0]
            }
    s=my_df.loc[str(ix[i].date())].fillna(dics)
    data=pd.concat([data,s])

In [35]:
len(data.loc['2015-01-14'])

25

In [36]:
data['close_prev_day']=data['close_prev_day'].shift(25)

In [37]:
data=data.dropna()

In [38]:
data.isnull().sum()

open              0
high              0
low               0
close             0
close_prev_day    0
fif_min_open      0
fif_min_close     0
fif_min_low       0
fif_min_high      0
dtype: int64

In [39]:
data.head()

,open,high,low,close,close_prev_day,fif_min_open,fif_min_close,fif_min_low,fif_min_high
date_time,,,,,,,,,
2015-01-12 09:15:00,18632.00,18632.00,18530.10,18560.45,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 09:30:00,18560.45,18646.85,18533.80,18646.85,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 09:45:00,18647.90,18737.10,18631.15,18729.95,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 10:00:00,18729.95,18755.75,18708.75,18748.15,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 10:15:00,18748.15,18787.70,18742.80,18765.95,18598.05,18632.0,18560.45,18530.1,18632.0


In [40]:
data=data.reset_index()

In [41]:
data['time']=data['date_time'].dt.time

In [48]:
data2=data.set_index('time')

In [50]:
ix2=data2.index

In [51]:
data2=data2.drop(ix2[0])

In [52]:
data2=data2.set_index('date_time', drop=False)

In [53]:
data2.head()

,date_time,open,high,low,close,close_prev_day,fif_min_open,fif_min_close,fif_min_low,fif_min_high
date_time,,,,,,,,,,
2015-01-12 09:30:00,2015-01-12 09:30:00,18560.45,18646.85,18533.80,18646.85,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 09:45:00,2015-01-12 09:45:00,18647.90,18737.10,18631.15,18729.95,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 10:00:00,2015-01-12 10:00:00,18729.95,18755.75,18708.75,18748.15,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 10:15:00,2015-01-12 10:15:00,18748.15,18787.70,18742.80,18765.95,18598.05,18632.0,18560.45,18530.1,18632.0
2015-01-12 10:30:00,2015-01-12 10:30:00,18763.15,18765.95,18728.95,18747.50,18598.05,18632.0,18560.45,18530.1,18632.0


In [54]:
data2['hour']=data2['date_time'].dt.hour
data2['min']=data2['date_time'].dt.minute

In [55]:
data2['call']=''

In [56]:
targ_l=[]
stp_l=[]
targ_s=[]
stp_s=[]
for i in range(len(data2)):
    targ_l.append(2*abs(data2['fif_min_close'][i]-data2['fif_min_open'][i])+data2['fif_min_close'][i])
    stp_l.append(data2['fif_min_close'][i]-abs(data2['fif_min_close'][i]-data2['fif_min_open'][i]))
    targ_s.append(data2['fif_min_close'][i]-2*abs(data2['fif_min_close'][i]-data2['fif_min_open'][i]))
    stp_s.append(data2['fif_min_close'][i]+abs(data2['fif_min_close'][i]-data2['fif_min_open'][i]))


In [57]:
data2['target_long']=targ_l
data2['stoploss_long']=stp_l
data2['target_short']=targ_s
data2['stoploss_short']=stp_s

In [64]:
data2.head()

,date_time,open,high,low,close,close_prev_day,fif_min_open,fif_min_close,fif_min_low,fif_min_high,hour,min,call,target_long,stoploss_long,target_short,stoploss_short
date_time,,,,,,,,,,,,,,,,,
2015-01-12 09:30:00,2015-01-12 09:30:00,18560.45,18646.85,18533.80,18646.85,18598.05,18632.0,18560.45,18530.1,18632.0,9,30,,18703.55,18488.9,18417.35,18632.0
2015-01-12 09:45:00,2015-01-12 09:45:00,18647.90,18737.10,18631.15,18729.95,18598.05,18632.0,18560.45,18530.1,18632.0,9,45,,18703.55,18488.9,18417.35,18632.0
2015-01-12 10:00:00,2015-01-12 10:00:00,18729.95,18755.75,18708.75,18748.15,18598.05,18632.0,18560.45,18530.1,18632.0,10,0,,18703.55,18488.9,18417.35,18632.0
2015-01-12 10:15:00,2015-01-12 10:15:00,18748.15,18787.70,18742.80,18765.95,18598.05,18632.0,18560.45,18530.1,18632.0,10,15,,18703.55,18488.9,18417.35,18632.0
2015-01-12 10:30:00,2015-01-12 10:30:00,18763.15,18765.95,18728.95,18747.50,18598.05,18632.0,18560.45,18530.1,18632.0,10,30,,18703.55,18488.9,18417.35,18632.0


In [62]:
data2.shape

(34776, 17)

In [65]:
data2['date']=data2['date_time'].dt.date

In [66]:
data2.head()

,date_time,open,high,low,close,close_prev_day,fif_min_open,fif_min_close,fif_min_low,fif_min_high,hour,min,call,target_long,stoploss_long,target_short,stoploss_short,date
date_time,,,,,,,,,,,,,,,,,,
2015-01-12 09:30:00,2015-01-12 09:30:00,18560.45,18646.85,18533.80,18646.85,18598.05,18632.0,18560.45,18530.1,18632.0,9,30,,18703.55,18488.9,18417.35,18632.0,2015-01-12
2015-01-12 09:45:00,2015-01-12 09:45:00,18647.90,18737.10,18631.15,18729.95,18598.05,18632.0,18560.45,18530.1,18632.0,9,45,,18703.55,18488.9,18417.35,18632.0,2015-01-12
2015-01-12 10:00:00,2015-01-12 10:00:00,18729.95,18755.75,18708.75,18748.15,18598.05,18632.0,18560.45,18530.1,18632.0,10,0,,18703.55,18488.9,18417.35,18632.0,2015-01-12
2015-01-12 10:15:00,2015-01-12 10:15:00,18748.15,18787.70,18742.80,18765.95,18598.05,18632.0,18560.45,18530.1,18632.0,10,15,,18703.55,18488.9,18417.35,18632.0,2015-01-12
2015-01-12 10:30:00,2015-01-12 10:30:00,18763.15,18765.95,18728.95,18747.50,18598.05,18632.0,18560.45,18530.1,18632.0,10,30,,18703.55,18488.9,18417.35,18632.0,2015-01-12


In [68]:
data2['date'][0]==data2['date'][25]

False

In [70]:
p=0
position=[]
for i in range(1,len(data2)):
    # giving buy call
    if(data2['fif_min_open'][i]>data2['close_prev_day'][i] and data2['open'][i]>data2['fif_min_high'][i] and p==0):
        position.append('buy')
        p=1
    elif(data2['open'][i]>data2['target_long'][i] and p==1):
        position.append('exit')
        p=2
    elif(data2['open'][i]<data2['stoploss_long'][i] and p==1):
        position.append('exit')
        p=2
        
    # giving sell call    
    elif(data2['fif_min_open'][i]<data2['close_prev_day'][i] and data2['open'][i]<data2['fif_min_low'][i] and p==0):
        position.append('sell')
        p=-1
    elif(data2['open'][i]<data2['target_short'][i] and p==-1):
        position.append('exit')
        p=2
    elif(data2['open'][i]>data2['stoploss_short'][i] and p==-1):
        position.append('exit')
        p=2
        
    
    elif(data2['hour'][i]>=15 and data2["min"][i]>=15 and (p==1 or p==-1)):
        position.append('exit')
        p=2
    elif(p==2 or p==0):
        position.append('No trade')
    elif(p==1):
        position.append('Buy_Hold')
    elif(p==-1):
        position.append('Sell_Hold')
    
    if(data2['date'][i-1]!=data2['date'][i]):
        p=0

In [71]:
position.insert(0,'No trade')

In [72]:
data2['call']=position

In [73]:
position=[]
for i in range(len(data2)):
    
    if(data2['call'][i]=='buy' or data2['call'][i]=='Buy_Hold'):
        position.append(1)
    elif(data2['call'][i]=='sell' or data2['call'][i]=='Sell_Hold'):
        position.append(-1)
    else:
        position.append(0)

In [74]:
data2['position']=position

In [75]:
data2.head()

,date_time,open,high,low,close,close_prev_day,fif_min_open,fif_min_close,fif_min_low,fif_min_high,hour,min,call,target_long,stoploss_long,target_short,stoploss_short,date,position
date_time,,,,,,,,,,,,,,,,,,,
2015-01-12 09:30:00,2015-01-12 09:30:00,18560.45,18646.85,18533.80,18646.85,18598.05,18632.0,18560.45,18530.1,18632.0,9,30,No trade,18703.55,18488.9,18417.35,18632.0,2015-01-12,0
2015-01-12 09:45:00,2015-01-12 09:45:00,18647.90,18737.10,18631.15,18729.95,18598.05,18632.0,18560.45,18530.1,18632.0,9,45,buy,18703.55,18488.9,18417.35,18632.0,2015-01-12,1
2015-01-12 10:00:00,2015-01-12 10:00:00,18729.95,18755.75,18708.75,18748.15,18598.05,18632.0,18560.45,18530.1,18632.0,10,0,exit,18703.55,18488.9,18417.35,18632.0,2015-01-12,0
2015-01-12 10:15:00,2015-01-12 10:15:00,18748.15,18787.70,18742.80,18765.95,18598.05,18632.0,18560.45,18530.1,18632.0,10,15,No trade,18703.55,18488.9,18417.35,18632.0,2015-01-12,0
2015-01-12 10:30:00,2015-01-12 10:30:00,18763.15,18765.95,18728.95,18747.50,18598.05,18632.0,18560.45,18530.1,18632.0,10,30,No trade,18703.55,18488.9,18417.35,18632.0,2015-01-12,0


In [76]:
data2['returns']=data2['open'].pct_change()

In [77]:
data2=data2.dropna()

In [78]:
data2["strategy"] = data2["position"].shift(1) * data2["returns"]
data2.dropna(inplace=True)
data2["creturns"] = (1+data2["returns"]).cumprod()-1
data2["cstrategy"] = (1+data2["strategy"]).cumprod()-1

# absolute performance of the strategy
perf = data2["cstrategy"].iloc[-1]

C:\Users\Aman\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Aman\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Aman\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [79]:
perf

-0.11775656627640918